# Donkey Colab with GPU

Donkey Colab with GPU.

# TensorFlowをVersion Down

Colab環境では、TensorFlowが1.13.0になっているので、1.12.0にDown gradeします。

Qiitaの記事を参考にDown gradeします。
https://qiita.com/a4_nghm/items/69cc5e6eaf07e9e09230

In [7]:
!apt-get --purge remove cuda*
!apt autoremove

!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-*/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-0

!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.12.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-cusparse-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-npp-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvprune-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvgraph-10-0' for glob 'cuda*'
Note, selecting 'cuda-cublas-dev-10-0' for glob 'cuda*'
Note, selecting 'cuda-gdb-10-0' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute--10-0' for glob 'cuda*'
Note, selecting 'cuda-visual-tools-10-0' for glob 'cuda*'
Note, selecting 'cuda-curand-dev-10-0' for glob 'cuda*'
Note, selecting 'cuda-cupti-10-0' for glob 'cuda*'
Note, selecting 'cuda-curand-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-libraries-dev-10-0' for glob 'cuda*'
Note, selecting 'cuda-cublas-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-repo-ubuntu1204' for glob 'cuda*'
Note, selecting 'cuda-nvjpeg-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvrtc-dev-10-0' for glob 'cuda*

# GPUの割り振りを確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [9]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# DonkeyレポジトリをGit clone

In [8]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6742 (delta 0), reused 0 (delta 0), pack-reused 6741
Receiving objects: 100% (6742/6742), 42.60 MiB | 22.79 MiB/s, done.
Resolving deltas: 100% (3887/3887), done.


# インストール

In [9]:
!pip3 install -e donkey

Obtaining file:///content/donkey
    100% |████████████████████████████████| 51kB 4.5MB/s 
    100% |████████████████████████████████| 225kB 14.8MB/s 
    100% |████████████████████████████████| 81kB 25.3MB/s 
    100% |████████████████████████████████| 194kB 29.2MB/s 
  Running setup.py develop for donkeycar


# Projectの作成

In [10]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


# datas.zipのアップロード
下記コードを実行すると、ファイルのアップロード フォームが出現します。datas.zipをアップリードします。

In [11]:
from google.colab import files
uploaded = files.upload()

Saving datas.zip to datas.zip


# datas.zipの移動と解答
datas.zipは、/content以下にアップされます。/content/mycar/dataに移動し、解凍します。

In [0]:
!mv /content/datas.zip /content/mycar/data/

In [13]:
%cd /content/mycar/data/

!unzip datas.zip

/content/mycar/data
Archive:  datas.zip
   creating: datas/
  inflating: datas/.DS_Store         
   creating: __MACOSX/
   creating: __MACOSX/datas/
  inflating: __MACOSX/datas/._.DS_Store  
   creating: datas/tub_02_19-01-31/
  inflating: datas/tub_02_19-01-31/1086_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/1202_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/955_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/1075_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/record_997.json  
  inflating: datas/tub_02_19-01-31/record_978.json  
  inflating: datas/tub_02_19-01-31/record_581.json  
  inflating: datas/tub_02_19-01-31/record_1118.json  
  inflating: datas/tub_02_19-01-31/1288_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/628_cam-image_array_.jpg  
  inflating: datas/tub_02_19-01-31/record_702.json  
  inflating: datas/tub_02_19-01-31/record_1231.json  
  inflating: datas/tub_02_19-01-31/record_352.json  
  inflating: datas/tub_

# 学習

学習する前に、ここまでインストールしたライブラリを有効にするために
**ランタイム-ランタイムの再起動**
を実行します。

In [5]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

tub_names None
train: 7241, validation: 1811
steps_per_epoch 56
Epoch 1/100
55/56 [============================>.] - ETA: 0s - loss: 39.6146 - angle_out_loss: 32.1129 - throttle_out_loss: 47.1163
Epoch 00001: val_loss improved from inf to 0.12540, saving model to /content/mycar/models/mypilot
56/56 [==============================] - 20s 352ms/step - loss: 38.9102 - angle_out_loss: 31.5441 - throttle_out_loss: 46.2764 - val_loss: 0.1254 - val_angle_out_loss: 0.2032 - val_throttle_out_loss: 0.0476
Epoch 2/100
55/56 [============================>.] - ETA: 0s - loss: 0.1288 - angle_out_loss: 0.1939 - throttle_out_loss: 0.0637
Epoch 00002: val_loss improved from 0.12540 to 0.10499, saving model to /content/mycar/models/mypilot
56/56 [==============================] - 15s 261ms/step - loss: 0.1284 - angle_out_loss: 0.1933 - throttle_out_loss: 0.0636 - val_loss: 0.1050 - val_angle_out_loss: 0.1489 - val_throttle_out_loss: 0.0611
Epoch 3/100
55/56 [============================>.] - ETA: 0s - l

KeyboardInterrupt: ignored

In [4]:
!nvidia-smi

Wed Feb 27 09:35:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  